In [ ]:
import math
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import matplotlib.cm as mcm
import modules.perps as perps
from modules.curves import DiscreteCurve
import csv

interest_rate = 0.1095
clamp_lower_bound = -0.0005
clamp_upper_bound = 0.0005

In [ ]:
markets = ["eth", "link", "sol"]
data_path = "./data/oracle_experiments/set_1/"

results = {}
key_cur_twap = "external TWAP (Chainlink)"
key_cur_twap_intern = "internal TWAP"
# additional columns
key_new_twap = "external TWAP (Uniswap)"
key_dif_twap = "external TWAP (Chainlink vs Uniswap, relative difference)"
key_rec_fund = "funding payment (Chainlink)"
key_rec_rate = "funding rate (Chainlink)"
key_new_fund = "funding payment (Uniswap)"
key_new_rate = "funding rate (Uniswap)"
key_rec_fund_no_clamp = "funding payment (Chainlink, no clamp)"
key_new_fund_no_clamp = "funding payment (Uniswap, no clamp)"
key_rec_rate_no_clamp = "funding rate (Chainlink, no clamp)"
key_new_rate_no_clamp = "funding rate (Uniswap, no clamp)"


for mkt in markets:
    df_periods = pd.read_csv(f"{data_path}/{mkt}_funding_periods.csv")
    df_periods = df_periods.rename(columns={"external TWAP":key_cur_twap})
    df_periods["start_time"]= pd.to_datetime(df_periods["start_time"])
    df_periods["end_time"]= pd.to_datetime(df_periods["end_time"])
    df_periods = df_periods.sort_values("start_time", ignore_index = True)
    
    funding_schedule = df_periods["start_time"].to_list()
    funding_schedule.append(df_periods.iloc[-1]["end_time"])

    df_oracle = pd.read_csv(f"{data_path}/{mkt}_oracle.csv")
    df_oracle["timestamp"]= pd.to_datetime(df_oracle["timestamp"])
    
    curve_schedule = np.array(df_oracle["timestamp"])
    curve = DiscreteCurve(dict(zip([x.timestamp() for x in curve_schedule], df_oracle.price)))

   
    df_periods[key_new_twap] = 0
    df_periods[key_rec_fund] = 0
    df_periods[key_new_fund] = 0
    df_periods[key_rec_rate] = np.nan
    df_periods[key_new_rate] = np.nan
    df_periods[key_rec_fund_no_clamp] = 0
    df_periods[key_new_fund_no_clamp] = 0
    df_periods[key_rec_rate_no_clamp] = np.nan
    df_periods[key_new_rate_no_clamp] = np.nan
    df_periods[key_dif_twap] = np.nan

    for i in range(0,len(funding_schedule)-1):
        cur_period =  df_periods.iloc[i]
        
        start = funding_schedule[i]
        end = funding_schedule[i+1]
        cur_internal_twap = cur_period[key_cur_twap_intern]
        cur_external_twap = cur_period[key_cur_twap]
        new_external_twap = perps.twap(curve, curve_schedule, start, end)*1e6
        
        delta_t = (end-start)/dt.timedelta(days=365)
        
        cur_funding_payment_recalc = perps.compute_funding_payment( 
            perp_twap = cur_internal_twap,
            spot_twap =  cur_external_twap,
            interest_rate = interest_rate,
            delta_t = delta_t,
            clamp_lower_bound = clamp_lower_bound,
            clamp_upper_bound = clamp_upper_bound)

        new_funding_payment_recalc = perps.compute_funding_payment( 
            perp_twap = cur_internal_twap,
            spot_twap =  new_external_twap,
            interest_rate = interest_rate,
            delta_t = delta_t,
            clamp_lower_bound = clamp_lower_bound,
            clamp_upper_bound = clamp_upper_bound)

        cur_funding_payment_recalc_no_clamp = perps.compute_funding_payment( 
            perp_twap = cur_internal_twap,
            spot_twap =  cur_external_twap,
            interest_rate = 0,
            delta_t = 0,
            clamp_lower_bound = 0,
            clamp_upper_bound = 0)

        new_funding_payment_recalc_no_clamp = perps.compute_funding_payment( 
            perp_twap = cur_internal_twap,
            spot_twap =  new_external_twap,
            interest_rate = 0,
            delta_t = 0,
            clamp_lower_bound = 0,
            clamp_upper_bound = 0)

        df_periods.loc[i,key_new_twap] = int(math.ceil(new_external_twap))
        df_periods.loc[i,key_rec_fund] = int(math.ceil(cur_funding_payment_recalc))
        df_periods.loc[i,key_new_fund] = int(math.ceil(new_funding_payment_recalc))
        df_periods.loc[i,key_rec_rate] = cur_funding_payment_recalc/cur_external_twap
        df_periods.loc[i,key_new_rate] = new_funding_payment_recalc/new_external_twap
        df_periods.loc[i,key_rec_fund_no_clamp] = int(math.ceil(cur_funding_payment_recalc_no_clamp))
        df_periods.loc[i,key_new_fund_no_clamp] = int(math.ceil(new_funding_payment_recalc_no_clamp))
        df_periods.loc[i,key_rec_rate_no_clamp] = cur_funding_payment_recalc_no_clamp/cur_external_twap
        df_periods.loc[i,key_new_rate_no_clamp] = new_funding_payment_recalc_no_clamp/new_external_twap
        df_periods.loc[i,key_dif_twap] = new_external_twap/cur_external_twap - 1

    results[mkt] = df_periods

In [ ]:
for mkt in markets:
    data = results[mkt]
    cols = 3
    fig, axes = plt.subplots(nrows=1, ncols=cols,figsize=(24, 6))
  
    data.plot.bar(ax=axes[0], x="end_time", y=[key_dif_twap])
    data.plot.bar(ax=axes[1], x="end_time", y=[key_rec_rate, key_new_rate])
    data.plot.bar(ax=axes[2], x="end_time", y=[key_rec_rate_no_clamp, key_new_rate_no_clamp])

    fig.suptitle(mkt)
    for i in range(cols):
        axes[i].yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
        axes[i].set_xlabel("funding period end")
    plt.savefig(f"{data_path}results_{mkt}.png", )


In [ ]:
# data_path = "./data/oracle_experiments/set_2"
# overlapping_samplings = {"30s_overlapping": 30, "300s_overlapping": 300}
# market = "sol"

# for key in overlapping_samplings:
#      df = pd.read_csv(f"{data_path}/{market}_twap_{key}.csv") 
#      with open(f"{data_path}/{market}_twap_{overlapping_samplings[key]}s.csv", 'w') as f:
#           writer = csv.writer(f)
#           writer.writerow(["timestamp", "price"])
#           last_ts = None
#           for index, row in df.iterrows():
#                ts = pd.to_datetime(row.timestamp)
#                if last_ts == None or pd.Timedelta(ts - last_ts).seconds >= overlapping_samplings[key]:
#                     writer.writerow([row.timestamp, row.price])
#                     last_ts = ts


In [ ]:
# data_path = "./data/oracle_experiments/set_2"

# time_deltas = [pd.Timedelta(5, "s"), pd.Timedelta(5, "m")]
# time_delta_col_name = lambda x: f"fallback_on_mid_after_{str(x).replace('0 days ', '')}"

# markets = ["eth", "sol"]
# for market in markets:
#     previous_mark_price = None
#     previous_mark_price_timestamp = None
#     file = f"{data_path}/{market}_market_data_vega.csv"
#     df = pd.read_csv(file) 

#     n = len(df.index)
#     for i in reversed(df.index):
#         current_timestamp = pd.to_datetime(df.loc[i,"timestamp"])
#         current_mark_price = df.loc[i,"mark_price"]
#         if current_mark_price != previous_mark_price:
#             previous_mark_price = current_mark_price
#             previous_mark_price_timestamp = current_timestamp

#         for delta in time_deltas:
#             col_name = time_delta_col_name(delta)
#             if current_timestamp - previous_mark_price_timestamp > delta:   
#                 df.loc[i,col_name] = df.loc[i,"mid_price"]
#             else:
#                 df.loc[i,col_name] = current_mark_price
#     df.to_csv(file, index=False)

   

In [ ]:
data_path = "./data/oracle_experiments/set_2/"

time_deltas = [pd.Timedelta(5, "s"), pd.Timedelta(5, "m")]
time_delta_col_name = lambda x: f"fallback_on_mid_after_{str(x).replace('0 days ', '')}"

markets = ["eth", "sol"]
start_limits = {"eth":  dt.datetime(year=2024, month=1, day=3, hour=0, tzinfo=dt.timezone.utc), "sol": dt.datetime(year=2024, month=1, day=4, hour=0, tzinfo=dt.timezone.utc)}
results = {}
internal_twap_keys = {}
external_twap_keys = {}
rate_keys = {}
for mkt in markets:
    start_limit = start_limits[mkt]
    end_limit = dt.datetime(year=2024, month=1, day=10, hour=0, tzinfo=dt.timezone.utc)

    samplings = ["15s", "30s", "300s"]
    additional_external_data = {"eth": ["ETHUSDT-1m-Binance-IndexPrice.csv"], "sol": ["SOLUSDT-1m-Binance-IndexPrice.csv"]}
    additional_internal_data = {"eth": ["ETHUSDT-1m-Binance-MarkPrice.csv", "ETHUSDT-1m-Binance-KLine.csv"], "sol": ["SOLUSDT-1m-Binance-MarkPrice.csv", "SOLUSDT-1m-Binance-KLine.csv"]}

    # additional column prefixes
    key_prefix_internal_twap = "internal TWAP"
    key_prefix_external_twap = "external TWAP"

    # load data
    df_periods = pd.read_csv(f"{data_path}/{mkt}_funding_rate_history.csv")
    df_periods["end_time"]= pd.to_datetime(df_periods["end_time"], utc=True, dayfirst=True)
    df_periods = df_periods.sort_values("end_time", ignore_index = True)

    df_vega_data = pd.read_csv(f"{data_path}/{mkt}_market_data_vega.csv")
    df_vega_data["timestamp"]= pd.to_datetime(df_vega_data["timestamp"])
    df_vega_data["next_MTM"]= pd.to_datetime(df_vega_data["next_MTM"])
    df_vega_data = df_vega_data.sort_values("timestamp", ignore_index = True)

    df_vega_oracle_data = pd.read_csv(f"{data_path}/{mkt}_chainlink_oracle_data.csv")
    df_vega_oracle_data["broadcast_at"] = pd.to_datetime(df_vega_oracle_data["broadcast_at"])
    df_vega_oracle_data = df_vega_oracle_data.sort_values("broadcast_at", ignore_index = True)

    keys_internal_twap = []
    keys_external_twap = []
    keys_rate = []

    internal_twap_columns = ["mark_price", "last_traded_price"]
    internal_twap_columns.extend([time_delta_col_name(delta) for delta in time_deltas])

    weighting_schemes = {"": 0, " (skewed weights)": 1}

    # loop through funding periods
    previous_index = None
    for i in df_periods.index:
        if previous_index == None:
            previous_index = i
            continue
        start = df_periods.loc[previous_index, "end_time"]
        end = df_periods.loc[i, "end_time"]
        df_periods.loc[i, "start_time"] = start
        previous_index = i
        if start_limit is not None and start < start_limit:
            continue
        if end_limit is not None and end > end_limit:
            continue

        delta_t = (end-start)/dt.timedelta(days=365)

        # internal TWAP calcs
        #   Binance
        for data in additional_internal_data[mkt]:
            df_additional_internal_data = pd.read_csv(f"{data_path}/{data}")
            df_additional_internal_data["open_time"] = pd.to_datetime([x*1e6 for x in df_additional_internal_data["open_time"]], utc = True)
            df_additional_internal_data = df_additional_internal_data.sort_values("open_time", ignore_index = True)
            
            binance_internal_curve_schedule = np.array(df_additional_internal_data["open_time"])
            binance_internal_curve = DiscreteCurve(dict(zip([x.timestamp() for x in binance_internal_curve_schedule], df_additional_internal_data["open"])))
            for k in weighting_schemes:
                binance_internal_twap = perps.twap(binance_internal_curve, binance_internal_curve_schedule, start, end, weighting_schemes[k])
                key_binance_internal_twap = f"{key_prefix_internal_twap} ({data.split('.csv')[0]}){k}"
                df_periods.loc[i,key_binance_internal_twap] = binance_internal_twap
                if not key_binance_internal_twap in keys_internal_twap:
                    keys_internal_twap.append(key_binance_internal_twap)

        #   Vega
        vega_curve_schedule = np.array(df_vega_data["next_MTM"])
        for col in internal_twap_columns:
            vega_curve = DiscreteCurve(dict(zip([x.timestamp() for x in vega_curve_schedule], df_vega_data[col])))
            for k in weighting_schemes:
                internal_twap = perps.twap(vega_curve, vega_curve_schedule, start, end, weighting_schemes[k])
                key_internal_twap = f"{key_prefix_internal_twap} ({col}){k}"
                df_periods.loc[i,key_internal_twap] = internal_twap
                if not key_internal_twap in keys_internal_twap:
                    keys_internal_twap.append(key_internal_twap)



        # external TWAP calcs
        #   Binance
        for data in additional_external_data[mkt]:
            df_additional_external_data = pd.read_csv(f"{data_path}/{data}")
            df_additional_external_data["open_time"] = pd.to_datetime([x*1e6 for x in df_additional_external_data["open_time"]], utc = True)
            df_additional_external_data = df_additional_external_data.sort_values("open_time", ignore_index = True)
            
            binance_external_curve_schedule = np.array(df_additional_external_data["open_time"])
            binance_external_curve = DiscreteCurve(dict(zip([x.timestamp() for x in binance_external_curve_schedule], df_additional_external_data["open"])))
            for k in weighting_schemes:
                binance_external_twap = perps.twap(binance_external_curve, binance_external_curve_schedule, start, end, weighting_schemes[k])
                key_binance_external_twap = f"{key_prefix_external_twap} ({data.split('.csv')[0]}){k}"
                df_periods.loc[i,key_binance_external_twap] = binance_external_twap
                if not key_binance_external_twap in keys_external_twap:
                    keys_external_twap.append(key_binance_external_twap)
        
        #   Chainlink
        chainlink_curve_schedule = np.array(df_vega_oracle_data["broadcast_at"])
        chainlink_curve = DiscreteCurve(dict(zip([x.timestamp() for x in chainlink_curve_schedule], [x*1e-8 for x in df_vega_oracle_data[f"{mkt}.price"]])))
        for k in weighting_schemes:
            chainlink_twap = perps.twap(chainlink_curve, chainlink_curve_schedule, start, end, weighting_schemes[k])
            key_chainlink_twap = f"{key_prefix_external_twap} (Chainlink){k}"
            df_periods.loc[i,key_chainlink_twap] = chainlink_twap
            if not key_chainlink_twap in keys_external_twap:
                keys_external_twap.append(key_chainlink_twap)

        #   Uniswap
        for sampling in samplings:
            df_oracle = pd.read_csv(f"{data_path}/{mkt}_twap_{sampling}.csv")
            df_oracle["timestamp"]= pd.to_datetime(df_oracle["timestamp"])
            curve_schedule = np.array(df_oracle["timestamp"])
            curve = DiscreteCurve(dict(zip([x.timestamp() for x in curve_schedule], df_oracle.price)))
            for k in weighting_schemes:
                sampling_external_twap = perps.twap(curve, curve_schedule, start, end, weighting_schemes[k])
                key_sampling_twap = f"{key_prefix_external_twap} (Uniswap, {sampling}){k}"
                df_periods.loc[i,key_sampling_twap] = sampling_external_twap
                if not key_sampling_twap in keys_external_twap:
                    keys_external_twap.append(key_sampling_twap)

        # funding rate calcs
        for internal_key in keys_internal_twap:
            internal_twap = df_periods.loc[i,internal_key] 
            for external_key in keys_external_twap:
                external_twap = df_periods.loc[i,external_key] 
                funding_payment = perps.compute_funding_payment( 
                    perp_twap = internal_twap,
                    spot_twap =  external_twap,
                    interest_rate = interest_rate,
                    delta_t = delta_t,
                    clamp_lower_bound = clamp_lower_bound,
                    clamp_upper_bound = clamp_upper_bound)
                key_rate = f"Vega rate ({internal_key}, {external_key})"
                df_periods.loc[i,key_rate] = funding_payment/external_twap
                if not key_rate in keys_rate:
                    keys_rate.append(key_rate)

    results[mkt] = df_periods.drop(0)
    internal_twap_keys[mkt] = keys_internal_twap.copy()
    external_twap_keys[mkt] = keys_external_twap.copy()
    rate_keys[mkt] = keys_rate.copy()

In [ ]:
mkt = "eth"
data = results[mkt].dropna()

keys_internal_twap = internal_twap_keys[mkt]
keys_external_twap = external_twap_keys[mkt]
keys_rate = rate_keys[mkt]

cols = 2
fig1, axes = plt.subplots(nrows=1, ncols=cols, figsize=(24, 6))
data.plot.bar(ax=axes[0], x="end_time", y=keys_internal_twap, cmap="Set1")
data.plot.bar(ax=axes[1], x="end_time", y=keys_external_twap, cmap="Set2")

commons_y_lims = {"eth": [2200,2450],"sol":[80,140]}
common_y_lim = commons_y_lims[mkt]

axes[0].set_ylim(common_y_lim)
axes[1].set_ylim(common_y_lim)

fig1.suptitle(f"{mkt}")
for i in range(cols):
    axes[i].legend(loc="upper left", ncol=2)
    axes[i].set_xlabel("funding period end")

fig2, ax = plt.subplots(figsize=(40, 10))
fig2.suptitle(f"{mkt}")

err = np.zeros([len(keys_internal_twap),len(keys_external_twap)])
for i in range(len(keys_internal_twap)):
    for j in range(len(keys_external_twap)):
        n = 0
        sq_err_sum = 0
        for k in data.index:
            start = data.loc[k, "start_time"]
            end = data.loc[k, "end_time"]
            delta_t = (end-start)/dt.timedelta(days=365)
            internal_twap = data.loc[k,keys_internal_twap[i]] 
            external_twap = data.loc[k,keys_external_twap[j]] 
            funding_payment = perps.compute_funding_payment( 
                perp_twap = internal_twap,
                spot_twap =  external_twap,
                interest_rate = interest_rate,
                delta_t = delta_t,
                clamp_lower_bound = clamp_lower_bound,
                clamp_upper_bound = clamp_upper_bound)
            rate = funding_payment/external_twap
            binance_rate = data.loc[k,"Binance"]
            n += 1
            sq_err_sum += math.pow(rate-binance_rate,2)
        err[i, j] = math.sqrt(sq_err_sum / n)

im = ax.imshow(err, cmap="Reds")
ax.set_xticks(np.arange(len(keys_external_twap)), labels=keys_external_twap)
ax.set_yticks(np.arange(len(keys_internal_twap)), labels=keys_internal_twap)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
for i in range(len(keys_internal_twap)):
    for j in range(len(keys_external_twap)):
        ax.text(j, i, f"{err[i, j]:.3E}", ha="center", va="center", color="black", size = 7)

ax.set_title("Root-mean-square error (vs Binance funding rate)")

fig3, ax = plt.subplots(nrows=1, ncols=1, figsize=(24, 6))
fig3.suptitle(f"{mkt}")

y = list(data.columns[1:11])
for k in weighting_schemes:
    y.append(f"Vega rate (internal TWAP (mark_price){k}, external TWAP (Chainlink){k})")
for k in weighting_schemes:
    y.append(f"Vega rate (internal TWAP (fallback_on_mid_after_00:05:00){k}, external TWAP (Uniswap, 300s){k})")

data.plot.bar(ax=ax, x="end_time", y=y, cmap="tab20")
ax.legend(loc="upper center", ncol=4)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

fig1.savefig(f"{data_path}results_{mkt}_fig1.png", )
fig2.savefig(f"{data_path}results_{mkt}_fig2.png", )
fig3.savefig(f"{data_path}results_{mkt}_fig3.png", )